# Natural Language Queries (NLQ) with Language Model on data

## Benefits - Answer questions based on natural language questions`

## How does it work?

![Langchain on SQL](https://python.langchain.com/assets/images/sql_usecase-d432701261f05ab69b38576093718cf3.png)

## Import libraries

In [1]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

## Setup sample database

In [2]:
import sqlite3

DATABASE_NM = "customer_complaints_db.sqlite"
conn = sqlite3.connect(DATABASE_NM) # create a database called customer_complaints_db
conn

## Create table customer_complaints

In [4]:
from sqlalchemy import create_engine
import pandas as pd

FILE_NAME = "credit_card_complaints.csv"
FILE_PATH = "/Users/sharattadimalla/github/nlq-app/public_data/"
pdf = pd.read_csv(FILE_PATH+FILE_NAME) # create pandas dataframe

pdf.head(5)

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2013-07-29,Credit card,NaN,Billing statement,NaN,NaN,NaN,Citibank,OH,45247,NaN,NaN,Referral,2013-07-30,Closed with explanation,True,True,469026
1,2013-07-29,Credit card,NaN,APR or interest rate,NaN,NaN,NaN,Synchrony Financial,WA,98548,NaN,NaN,Web,2013-07-29,Closed with monetary relief,True,False,469131
2,2013-07-29,Credit card,NaN,Delinquent account,NaN,NaN,NaN,Amex,TX,78232,NaN,NaN,Web,2013-07-29,Closed with monetary relief,True,False,479990
3,2013-07-29,Credit card,NaN,Billing disputes,NaN,NaN,NaN,Capital One,FL,32226,Servicemember,NaN,Web,2013-07-29,Closed with explanation,True,False,475777
4,2013-07-29,Credit card,NaN,Credit line increase/decrease,NaN,NaN,NaN,Citibank,WI,53066,Older American,NaN,Phone,2013-07-30,Closed with explanation,True,True,469473


## Get customer_complaints DDL from pandas df

In [5]:
ddl_sql = pd.io.sql.get_schema(pdf.reset_index(), 'customer_complaints')
print(ddl_sql)

CREATE TABLE "customer_complaints" (
"index" INTEGER,
  "date_received" TEXT,
  "product" TEXT,
  "sub_product" REAL,
  "issue" TEXT,
  "sub_issue" REAL,
  "consumer_complaint_narrative" TEXT,
  "company_public_response" TEXT,
  "company" TEXT,
  "state" TEXT,
  "zip_code" TEXT,
  "tags" TEXT,
  "consumer_consent_provided" TEXT,
  "submitted_via" TEXT,
  "date_sent_to_company" TEXT,
  "company_response_to_consumer" TEXT,
  "timely_response" INTEGER,
  "consumer_disputed" INTEGER,
  "complaint_id" INTEGER
)


In [6]:
ddl_sql ="""CREATE TABLE IF NOT EXISTS "customer_complaints" (
  "index" INTEGER,
  "date_received" TEXT,
  "product" TEXT,
  "sub_product" REAL,
  "issue" TEXT,
  "sub_issue" REAL,
  "consumer_complaint_narrative" TEXT,
  "company_public_response" TEXT,
  "company" TEXT,
  "state" TEXT,
  "zip_code" TEXT,
  "tags" TEXT,
  "consumer_consent_provided" TEXT,
  "submitted_via" TEXT,
  "date_sent_to_company" TEXT,
  "company_response_to_consumer" TEXT,
  "timely_response" INTEGER,
  "consumer_disputed" INTEGER,
  "complaint_id" INTEGER
)"""

In [7]:
TABLE_NM = 'customer_complaints'

cur = conn.cursor()

cur.execute(ddl_sql)

conn.commit()

table_exists_sql = "SELECT name FROM sqlite_master WHERE type='table' AND name='{customer_complaints}';"

conn.execute(table_exists_sql).fetchall()

pdf.to_sql(TABLE_NM, conn, if_exists='append', index=False) # create table in db


87718

## Verify from table

In [8]:
customer_complaints_count_sql = "select count(*) from customer_complaints;"
conn.execute(customer_complaints_count_sql).fetchall()

[(87718,)]

## Stitch LangChain

In [9]:
import dotenv
dotenv.load_dotenv()

True

In [10]:
db = SQLDatabase.from_uri("sqlite:///customer_complaints_db.sqlite", 
                          include_tables=[
                            "customer_complaints"
                            ],
                          sample_rows_in_table_info=2
                         )
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

## Ask Natural Language Queries to the db_chain

In [11]:
question1 = "How many customer complaints are there?"
db_chain.run(question1)



> Entering new SQLDatabaseChain chain...
How many customer complaints are there?
SQLQuery:SELECT COUNT(*) FROM customer_complaints;
SQLResult: [(87718,)]
Answer:There are 87718 customer complaints.
> Finished chain.


'There are 87718 customer complaints.'

In [12]:
question2 = "What are the top 5 issues?"
db_chain.run(question2)



> Entering new SQLDatabaseChain chain...
What are the top 5 issues?
SQLQuery:SELECT issue, COUNT(*) AS count FROM customer_complaints GROUP BY issue ORDER BY count DESC LIMIT 5;
SQLResult: [('Billing disputes', 14827), ('Other', 9144), ('Identity theft / Fraud / Embezzlement', 8330), ('Closing/Cancelling account', 6291), ('APR or interest rate', 5463)]
Answer:The top 5 issues are Billing disputes, Other, Identity theft / Fraud / Embezzlement, Closing/Cancelling account, and APR or interest rate.
> Finished chain.


'The top 5 issues are Billing disputes, Other, Identity theft / Fraud / Embezzlement, Closing/Cancelling account, and APR or interest rate.'

In [13]:
question3 = "which companies had the top 5 complaints?"
db_chain.run(question3)



> Entering new SQLDatabaseChain chain...
which companies had the top 5 complaints?
SQLQuery:SELECT company, COUNT(*) AS num_complaints FROM customer_complaints GROUP BY company ORDER BY num_complaints DESC LIMIT 5;
SQLResult: [('Citibank', 16561), ('Capital One', 12740), ('JPMorgan Chase & Co.', 10203), ('Bank of America', 8995), ('Synchrony Financial', 8637)]
Answer:The top 5 companies with the most complaints are Citibank, Capital One, JPMorgan Chase & Co., Bank of America, and Synchrony Financial.
> Finished chain.


'The top 5 companies with the most complaints are Citibank, Capital One, JPMorgan Chase & Co., Bank of America, and Synchrony Financial.'

In [14]:
question4 = "How many issues were resolved in timely fashion?"
db_chain.run(question4)



> Entering new SQLDatabaseChain chain...
How many issues were resolved in timely fashion?
SQLQuery:SELECT COUNT(*) FROM customer_complaints WHERE timely_response = 1;
SQLResult: [(86750,)]
Answer:86750 issues were resolved in timely fashion.
> Finished chain.


'86750 issues were resolved in timely fashion.'

In [15]:
question5 = "what are the top 5 channels by which complaints are submmited?"
db_chain.run(question5)



> Entering new SQLDatabaseChain chain...
what are the top 5 channels by which complaints are submmited?
SQLQuery:SELECT submitted_via, COUNT(*) AS count FROM customer_complaints GROUP BY submitted_via ORDER BY count DESC LIMIT 5;
SQLResult: [('Web', 60384), ('Referral', 14565), ('Phone', 6587), ('Postal mail', 5257), ('Fax', 881)]
Answer:The top 5 channels by which complaints are submitted are Web, Referral, Phone, Postal mail, and Fax.
> Finished chain.


'The top 5 channels by which complaints are submitted are Web, Referral, Phone, Postal mail, and Fax.'